In [2]:
import os
import glob
import json
import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors

import sys
sys.path.append(os.path.abspath(".."))
from function import ART_downscale as ART_down

## Code to export Yearly OBS Wet days and Weibull Parameters

In [3]:
OBS_base = os.path.join('/','media','arturo','Arturo','Data','Italy')

In [4]:
veneto_dir = os.path.join('/','media','arturo','Arturo','Data','shapes','Europa','Italy')

if os.path.exists(veneto_dir):
    VENETO = gpd.read_file(os.path.join(veneto_dir,'Veneto.geojson'))
    DEM = gpd.read_file(os.path.join(veneto_dir,'Veneto_DEM_500.geojson'))
else:
    raise SystemExit(f"File not found: {veneto_dir}")

In [5]:
OBS_base = os.path.join(OBS_base,'stations','data')
OBS_INFO_dir = os.path.join(os.path.join(OBS_base),'Veneto','BACKUP_RAW','VENETO_INFO_1dy.csv')

if os.path.exists(OBS_INFO_dir):
    INFO_pd = pd.read_csv(OBS_INFO_dir, sep=',')
else:
    raise SystemExit(f"File not found: {OBS_INFO_dir}")

## Create a list of files with start and end years

In [6]:
names, lats, lons, yys, yye, nys, elev, Ns, Cs, Ws = [], [], [], [], [], [], [], [], [], []

for nn in range(len(INFO_pd)):
    RAIN_pd = pd.read_csv(os.path.join(OBS_base,'Veneto','BACKUP_RAW','CLEAR_1dy',f'{INFO_pd['File_Name'].values[nn]}.csv'),sep=',')
    RAIN_pd['TIME'] = pd.to_datetime(RAIN_pd['TIME'].values)
    ss, ee = RAIN_pd['TIME'][0].year, RAIN_pd['TIME'][len(RAIN_pd['TIME'])-1].year
    ny = (ee-ss)+1

    yys_new = pd.to_datetime(RAIN_pd['TIME'].values[0]).year
    yye_new = pd.to_datetime(RAIN_pd['TIME'].values[-1]).year

    RAIN_xr = xr.DataArray(RAIN_pd['PRE'].values,
                        coords={'time':RAIN_pd['TIME'].values}, 
                        dims=('time'))

    RAIN_pd = RAIN_pd.dropna()

    names.append(INFO_pd['File_Name'].values[nn])
    lats.append(INFO_pd['Lat'].values[nn])
    lons.append(INFO_pd['Lon'].values[nn])
    yys.append(ss)
    yye.append(ee)
    nys.append(ny)
    elev.append(INFO_pd['Elv'].values[nn])

wa_pd = pd.DataFrame({'File_Name':names, 
                    'Lat':lats, 'Lon':lons, 
                    'YYS':yys, 'YYE':yye, 
                    'Elv':elev,'NY':nys})

nameout = os.path.join(OBS_base,'Weibull','Veneto_INFO.csv')
wa_pd.to_csv(nameout,index=False,header=True)

In [7]:
wa_pd.head(5)

,File_Name,Lat,Lon,YYS,YYE,Elv,NY
0,002_FG_BS,45.895600,12.815000,1995,2022,21,28
1,003_BL_Ar,46.499984,11.876073,1984,2023,1642,40
2,003_FG_BS,45.918000,12.545000,1995,2022,22,28
3,004-264_NP_Be,46.164679,12.244708,1993,2023,379,31
4,004_NP_Be,46.133632,12.195395,1993,2006,392,14


## Compute and Export Wet days and Weibull parameter for each year

In [29]:
names, lats, lons = [], [], []
for nn in range(len(wa_pd)):
    filename = f'{wa_pd['File_Name'].values[nn]}.csv'
    RAIN_pd = pd.read_csv(os.path.join(OBS_base,'Veneto','BACKUP_RAW','CLEAR_1dy', filename), sep=',')
    RAIN_pd['TIME'] = pd.to_datetime(RAIN_pd['TIME'].values)
    ss, ee = RAIN_pd['TIME'][0].year, RAIN_pd['TIME'][len(RAIN_pd['TIME'])-1].year
    ny = (ee-ss)+1
    years = np.arange(ss,ee+1)

    if ny != len(years):
        raise SystemExit(f"Problems with the years")

    RAIN_xr = xr.DataArray(
                RAIN_pd['PRE'].values,
                coords={'time':RAIN_pd['TIME'].values}, 
                dims=('time'))

    NCW = ART_down.fit_yearly_weibull_update(RAIN_xr,1,40) # 40 maximum miss data
    N = NCW[:,0]
    C = NCW[:,1]
    W = NCW[:,2]

    Weibull = pd.DataFrame({
                'Year':years,
                'N':N,
                'C':C,
                'W':W
    })

    nameout = os.path.join(OBS_base,'Weibull','Veneto',filename)
    Weibull.to_csv(nameout,index=False,header=True)

Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enough data
Not enou